<a href="https://colab.research.google.com/github/barbaroja2000/rubric/blob/main/Rubric_OpenAI_GPT_3_5_Turbo_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluating OpenAI GPT 3.5 against a custom rubric with GPT-4

Uses:

*   Langsmith for custom rubrik datasets & Evaluation Framework
*   GPT-4 To evaluate LLM output
*   gpt-3.5-turbo-1106 Model from OpenAI

---

Test:

*  Provides a quality assesment 0-5 and a rationalle for the score
* The interactive table can be used to Check the model output against the GPT-4 assesment
* Possible improvment here is to be able to manually update the score returned from the model

---

Model gpt-3.5-turbo-1106:


---

Refs:

https://platform.openai.com/docs/models/gpt-3-5

https://python.langchain.com/docs/langsmith/

---

Requires:
*   Test dataset in Langsmith: This notebook shows how to set this up https://github.com/barbaroja2000/rubric/blob/main/Langsmith_Rubric_Dataset_Creator.ipynb
*   Langchain Key
*   OpenAI Key


In [ ]:
!pip install -qU langchain langsmith openai tiktoken

In [ ]:
from google.colab import userdata
import os
import requests

os.environ['LANGCHAIN_ENDPOINT']= "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']= userdata.get('langchain_api_key')
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

In [ ]:
project_name  = "OpenAI GPT 3.5 Turbo Chat"
rubric_model = "gpt-3.5-turbo-1106"
rubric_temperature=0
evaluator_model="gpt-4-1106-preview"
evaluator_temperature=0
dataset_name = "General Rubric: Extended"

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
rubric_llm = ChatOpenAI(model=rubric_model, temperature=rubric_temperature)

In [ ]:
from langsmith import Client
client = Client()

In [ ]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# create a string template for a System role
system_template = """
Please provide a detailed, accurate, and contextually relevant response to the following query.
Your answer should demonstrate a clear understanding of the subject matter, considering any logical,
ethical, historical, or cultural aspects involved. Ensure the response is tailored to the specific
requirements of the question, whether it involves analysis, explanation, creativity, or problem-solving.
"""

In [ ]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap, RunnablePassthrough


def create_runnable():
    prompt = ChatPromptTemplate.from_messages([
      ("system", system_template),
      ("human", "{input}")
    ])
    return RunnableMap({"input": RunnablePassthrough()}) | prompt | rubric_llm

In [ ]:
import re
from typing import Any, Optional

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import StringEvaluator

class QualityEvaluator(StringEvaluator):
    """An LLM-based quality  evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model=evaluator_model, temperature=evaluator_temperature)

        template = """ Evaluate the answer provided from 0 to 5, with 5 being the best quality answer, and 0 where the answer is false or not provided
        --------
        QUESTION: {input}
        --------
        ANSWER: {prediction}
        --------
        Consisely reason step by step about why the given score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = LLMChain.from_string(llm=llm, template=template)

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "quality"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain(
            dict(input=input, prediction=prediction), **kwargs
        )
        lines = evaluator_result["text"].strip().split("\n")
        reasoning = evaluator_result["text"].strip()
        score = lines[-1]
        if score is not None:
            score = float(score.strip())
        return {"score": score, "reasoning": reasoning}

In [ ]:
import random
import string

def rando():
  # Generating a random 4-letter string
  random_string = ''.join(random.choices(string.ascii_letters, k=4))
  return random_string

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    input_key="question",
    output_key="output",
    custom_evaluators = [QualityEvaluator()],
)

project_metadata = {
  rubric_model : rubric_model,
  rubric_temperature : rubric_temperature,
  evaluator_model : evaluator_model,
  evaluator_temperature: evaluator_temperature
}

run = run_on_dataset(
    client=client,
    project_metadata = project_metadata,
    project_name = f"{project_name}-{rando()}",
    dataset_name=dataset_name,
    llm_or_chain_factory=create_runnable,
    evaluation=evaluation_config,
    concurrency_level=1, #This will bork if not provided or set to anything other than 1
)

View the evaluation results for project 'OpenAI GPT 3.5 Turbo Chat-fWqN' at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd/compare?selectedSessions=df92dfaa-b12a-46de-b2a8-db86ebc8f34d

View all tests for Dataset General Rubric: Extended at:
https://smith.langchain.com/o/8bebdbff-9433-40f6-b7e1-f013521a8100/datasets/dfedd53c-204a-4b69-9f64-51a61568a7fd
[----------------------------->                    ] 73/123

In [ ]:
# Extracting the required information
extracted_data = []
for key, value in run['results'].items():
    output = value["output"] if "output" in value else None
    question = value['input']['question']
    for feedback_item in value['feedback']:
        if feedback_item.key == 'quality':
            quality = feedback_item.score
            feedback = feedback_item.comment
            extracted_data.append({
                'question': question,
                'output': output,
                'quality': quality,
                'feedback': feedback
            })

In [ ]:
import pandas as pd
from google.colab import data_table
df = pd.DataFrame(extracted_data)

data_table.DataTable(df, include_index=False, num_rows_per_page=10)

,question,output,quality,feedback
0,"I WAS born in the year 1632, in the city of Yo...",content='The passage is an excerpt from Daniel...,4.0,The provided answer offers a succinct summary ...
1,"I WAS born in the year 1632, in the city of Yo...","content=""- The narrator, Robinson Crusoe, was ...",5.0,Evaluation of the Answer:\n\n- The answer corr...
2,"Y curiosity, in a sense, was stronger than my ...",content='The passage provided is an excerpt fr...,4.0,The provided answer does a good job of identif...
3,Explain the role of a central bank in an econo...,content='A central bank plays a crucial role i...,5.0,The answer provided is comprehensive and expla...
4,Describe the basics of how the stock market wo...,"content=""Sure! The stock market is a place whe...",5.0,The answer provided is comprehensive and expla...
...,...,...,...,...
118,Analyze the emotional content of this diary en...,"content='The diary entry ""Today was an ordinar...",5.0,The provided answer offers a thorough analysis...
119,What is the emotional tone of this speech exce...,content='The emotional tone of the speech exce...,5.0,The provided answer accurately identifies the ...
120,Determine the emotional tone of this message: ...,"content='The emotional tone of the message ""I ...",5.0,The provided answer does an excellent job of a...
121,Analyze the emotion in this customer review: '...,"content='The customer review ""I\'ve never been...",5.0,The provided answer does an excellent job of a...


In [ ]:
pd.to_numeric(df.quality).mean()

4.729674796747967

In [ ]:
examples = list(client.list_examples(dataset_name=dataset_name))

In [ ]:
# Creating a new DataFrame from the list
list_df = pd.DataFrame([{'question': item.inputs['question'], 'category': item.inputs['category']} for item in examples])

In [ ]:
# Merging the DataFrames
merged_df = pd.merge(df, list_df, on='question')

In [ ]:
# Group by 'category' and calculate the mean of 'quality' scores
avg_scores_per_category = merged_df.groupby('category')['quality'].mean()

print(avg_scores_per_category)

category
Categorization                           4.863636
Coding                                   4.673077
Creative Writing                         4.520833
Cultural and Contextual Understanding    5.000000
Emotion Analysis                         4.700000
Ethics Analysis                          4.888889
Fact Analysis                            5.000000
Logical Reasoning                        4.545455
Reading Comprehension                    4.666667
Reframing                                4.562500
Safety and Security                      4.750000
Summarization                            4.333333
Name: quality, dtype: float64
